In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession.builder.getOrCreate()

import pyspark.sql.functions as f


In [ ]:
data = spark.read.format("csv").load("../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv", header = True)

In [ ]:
display(data)

In [ ]:
data.show(20)

In [ ]:
print(data.count())

In [ ]:
selected_data = data.select("id", "gender", "age", "stroke")
display(selected_data)

In [ ]:
selected_data = data.selectExpr("id as ID", "age as Age", "stroke as Stroke", "gender")
display(selected_data)

In [ ]:
selected_data.show(10)

In [ ]:
print(selected_data.select("ID").count())

In [ ]:
selected_data.select("Stroke").distinct().show()

In [ ]:
dropDisDF = selected_data.dropDuplicates(["Stroke"])
dropDisDF.select("Stroke").show()
print("Distinct count of department & salary : "+str(dropDisDF.count()))


In [ ]:
selected_data.filter("Stroke == 1").show(10)

In [ ]:
print(selected_data.filter("Gender == 'Female'").filter("Stroke == 1").count())
print(selected_data.filter("Gender == 'Male'").filter("Stroke == 1").count())

In [ ]:
num_females = selected_data.filter("Gender == 'Female'").count()
num_males = selected_data.filter("Gender == 'Male'").count()
num_females_withstroke = selected_data.filter("Gender == 'Female'").filter("Stroke == 1").count()
num_males_withstroke = selected_data.filter("Gender == 'Male'").filter("Stroke == 1").count()
print(num_females_withstroke*100/num_females)
print(num_males_withstroke*100/num_males)

In [ ]:
print(selected_data.filter(selected_data.Age.isin([80,81,82])).count())

df = selected_data.filter(f.col("Age").isin([80,81,82]))
df.show()

In [ ]:
selected_data.orderBy("Age").show()

In [ ]:
selected_data.orderBy(selected_data.Age.asc()).show(truncate=False)

selected_data.orderBy(f.desc("Age")).show()

In [ ]:
selected_data.orderBy(selected_data.Age.desc()).show(truncate=False) 

selected_data.withColumn("gender_lower", f.lower(f.col("Gender"))).show()

In [ ]:
selected_data = selected_data.drop("gender_lower")
selected_data.show()

In [ ]:
selected_data.orderBy(f.desc("Age")).show()

In [ ]:
import pyspark.sql.functions as f
import pyspark.sql.types as t
selected_data = selected_data.withColumn("Age", f.col("Age").cast(t.IntegerType())).withColumn("Stroke", f.col("Stroke").cast(t.IntegerType()))
selected_data.show()

In [ ]:
selected_data = selected_data.withColumn("gender_dummy",f.when(f.col("Gender") == 'Female', 1).otherwise(0))
selected_data.show()

In [ ]:
sum_stroke = selected_data.groupBy("Gender").agg(f.sum(f.col("Stroke")).alias("sum_stroke"))

sum_stroke.show()
sum_stroke2 = selected_data.agg(f.sum(f.col("Stroke")).alias("sum_stroke"))
sum_stroke2.show()
selected_data.count()

In [ ]:
selected_data.groupBy("Gender").agg(f.max(f.col("Age"))).show()

In [ ]:
selected_data.groupBy("Gender").agg(f.min(f.col("Age"))).show()

In [ ]:
agg_data = selected_data.groupBy("Gender").agg(f.collect_list(f.col("Stroke")).alias("Stroke_list_per_gender"))
agg_data.show()

In [ ]:
selected_data.groupBy("Gender").agg(f.avg("Stroke").alias("Average Stroke Rate")).show()